# Analiza głównych składowych

Zbiór danych: https://www.kaggle.com/datasets/teejmahal20/airline-passenger-satisfaction?select=train.csv

Cel biznesowy: 
- Identyfikacja niezadowolonych klientów z lotu.
- Aktywna kampania dla niezadowolonych klientów, w celu zachęcenia do ponownego skorzystania z usług.


Etapy demo:
1. Załadowanie wymaganych bibliotek
2. Wczytanie i przegląd danych
3. Standaryzacja
4. Analiza głównych składowych
5. Porównanie modelu drzewa decyzyjnego na podstawie PCA i zmiennych surowych

### 1. Załadowanie wymaganych bibliotek

In [ ]:
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

### 2. Wczytanie i przegląd danych

In [ ]:
df = pd.read_csv('data/flight_satisfaction.csv')
del df['Unnamed: 0']

In [ ]:
# nagłówek
df.head()

In [ ]:
# liczebność zmiennej celu
df['satisfaction_satisfied'].value_counts()

In [ ]:
# przypisanie targetu do zmiennej
y = df['satisfaction_satisfied']
del df['satisfaction_satisfied']

### 3. Standaryzacja

In [ ]:
df_scaled  = StandardScaler().fit_transform(df)
df_scaled = pd.DataFrame(df_scaled, columns = df.columns)
df_scaled

In [ ]:
# Sprawdzenie wyników standaryzacji
df_scaled.describe()

## 4. Analiza Głównych składowych (PCA)

In [ ]:
# model PCA
pca  = PCA().fit(...)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
# Wizualizacja
plt.plot(pca.explained_variance_ratio_)
plt.title('Explained variance ratio')
plt.show()

In [ ]:
# Zbiór PCA
cols =  ['V'+str(i) for i in range(len(df_scaled.columns))]
df_pca = pd.DataFrame(pca.transform(df_scaled),columns=cols)

In [ ]:
# Nagłówek
df_pca.head()

In [ ]:
# Korelacja
df_pca.corr()

## 5. Porównanie modelu drzewa decyzyjnego na podstawie PCA i zmiennych surowych

In [ ]:
# Wytypowanie głównych składowych do modelu
x_names = cols[:7]
x_names

In [ ]:
# dodanie zmiennej y
df_pca['satisfied'] = y 

In [ ]:
# train/test split
train_x_pca, test_x_pca, train_y_pca, test_y_pca = train_test_split(df_pca[x_names], df_pca['satisfied'], random_state=123)

In [ ]:
# train/test split
train_x, test_x, train_y, test_y = train_test_split(df, y, random_state=123)

In [ ]:
# model ze zmiennymi PCA
model_pca = DecisionTreeClassifier(max_depth=10, min_samples_split=20).fit(...)

In [ ]:
# model standardowy
model_std = DecisionTreeClassifier(max_depth=10, min_samples_split=20).fit(...)

In [ ]:
# Accuracy - model PCA test
model_pca.score(...)

In [ ]:
# Accuracy - model PCA train
model_pca.score(...)

In [ ]:
# Accuracy - model standardowy test
model_std.score(...)

In [ ]:
# Accuracy - model standardowy train
model_std.score(...)